# Hn Stage to OMOP

Hn Stage is mainly mapped to OMOP Measurement, following the mapping described in: https://docs.google.com/spreadsheets/d/1Vw1Dr2K4oG__cDQTutGaJhZvGUvQTLwc4qWreP6qMSs/edit?gid=1518421257#gid=1518421257



In [ ]:
import mysql.connector
import pandas as pd
import psycopg2
import random
import numpy as np
from datetime import datetime, timedelta

In [ ]:
hnStage_IDEA4RC = pd.read_csv("./IDEA4RC-data/hnStageIDEA4RC.csv")
hnStage_IDEA4RC.head(5)

In [ ]:
conn = psycopg2.connect(
    dbname="omopdb",
    user="postgres",
    password="mysecretpassword",
    host="localhost",
    port="5432"
)

cur = conn.cursor()
config = {
    'user': 'user', 
    'password': 'password',
    'host': '127.0.0.1',
    'database': 'idea4rc_dm',
    'raise_on_warnings': True
}

conn2 = mysql.connector.connect(**config)
curIDEA = conn2.cursor()

### Hn Stage to Measurement

For the following:

Soft tissue
distant lymph node
lung
bone
liver
pleura
peritoneum
brain
other viscera
unknown

We will be ignoring the variable as modifier since it does not give us real information

In [ ]:
df_tables=hnStage_IDEA4RC

sql = """
    INSERT INTO omopcdm.measurement (person_id, measurement_concept_id, measurement_date,measurement_type_concept_id,value_as_concept_id,measurement_event_id, meas_event_field_concept_id)
    VALUES (%s, %s, %s,%s,%s,%s,%s)
"""
cols=['bone', 'brain', 'distant_lymph_node', 'liver', 'lung', 'other_viscera', 
 'peritoneum', 'pleura', 'soft_tissue', 'unknown', 'episode_event', 
 'cm', 'cn', 'ct', 'pm', 'pn', 'pt', 'r_ene', 'staging_classification']

query= """
    SELECT c.patient 
    FROM cancer_episode c
    WHERE EXISTS (
        SELECT 1 
        FROM EpisodeEvent e
        WHERE e.cancerEpisode = c.id
        AND e.id = %s
    )
    LIMIT 1

"""
queryDate="""
    SELECT episode_start_date
    FROM cancer_episode
    WHERE id = %s
"""
sqlGetEpisode = """
    SELECT c.id 
    FROM episode c
    WHERE c.person_id = %s
    AND c.episode_start_date = %s
    LIMIT 1
"""

booleans=[
    "Soft tissue",
    "distant lymph node",
    "lung",
    "bone",
    "liver",
    "pleura",
    "peritoneum",
    "brain",
    "other viscera",
    "unknown"
]
boolValues=[4188539,4188540]
meas_event_field_concept_id=798885
date=datetime.now()
measurement_type_concept_id=38000280

for idx, row in df_tables.iterrows():
    for column in cols:
        curIDEA.execute(query, (row['Episode Event Reference']))
        resIDEA=cur.fetchone()
        person_id=res
        measurement_value = row[column]
        if((column in booleans) and (measurement_value not in boolValues)):
            if measurement_value==0:
                measurement_value=4188540
            else:
                measurement_value=4188539
        measurement_concept_id = row[column]
        curIDEA.execute(queryDate, (row['Episode Event Reference']))
        res=curIDEA.fetchone()
        dateAux=res
        cur.execute(sqlGetEpisode, (person_id,dateAux))
        measurement_event_id=cur.fetchone()
        cur.execute(sql, (person_id, measurement_concept_id, date, measurement_type_concept_id,measurement_value,measurement_event_id,meas_event_field_concept_id))
        conn.commit()



In [ ]:
#1147138 is the ID of the column `episode_event_field_concept_id` in `episode_event`, which will guide us to the appropriate column in `measurement`.